In [1]:
import sys
from pathlib import Path

In [4]:
!{sys.executable} -m pip install -e ..

Obtaining file:///home/ajkdrag/workspace/chequeparser
  Preparing metadata (setup.py) ... done
  Attempting uninstall: chequeparser
    Found existing installation: chequeparser 0.0.1
    Uninstalling chequeparser-0.0.1:
      Successfully uninstalled chequeparser-0.0.1
  Running setup.py develop for chequeparser


## imports

In [5]:
from chequeparser.datasets.fileds import FileDS

ModuleNotFoundError: No module named 'chequeparser'

## config